<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [3]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [11]:
support.Unzip('/content/LABELS_V3.0.zip','/content/Pests/Labels')

In [ ]:
support.fileCount('/content/Pests/Labels')

In [13]:
dataSet = Imagedataset.createImageDataset()

In [14]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object ,32, '/content/Pests/Images/',  test_size=11241 )

In [ ]:
listdir('/content/Pests/Labels')

In [16]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [48]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44964, 32, 32, 3)
(44964, 10)
(11241, 32, 32, 3)
(11241, 10)


In [47]:
import numpy as np 
trainX = np.load('/content/drive/My Drive/Research-Data/trainX.npy') 
trainY = np.load('/content/drive/My Drive/Research-Data/trainY.npy') 
testX = np.load('/content/drive/My Drive/Research-Data/testX.npy') 
testY = np.load('/content/drive/My Drive/Research-Data/testY.npy') 

In [45]:
import numpy as np 
np.save('/content/drive/My Drive/Research-Data/trainX',trainX) 
np.save('/content/drive/My Drive/Research-Data/trainY',trainY) 
np.save('/content/drive/My Drive/Research-Data/testX',testX) 
np.save('/content/drive/My Drive/Research-Data/testY', testY) 

In [18]:
import tensorflow as tf
from keras import models
from keras.applications.vgg16 import VGG16
vgg = VGG16(input_shape=[32,32,3], include_top=False, pooling='avg',weights='imagenet')
vgg.summary()

Using TensorFlow backend.


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [21]:
from keras.models import Model
layerName = 'block1_pool'
mymodel = Model(inputs= vgg.input , outputs=vgg.get_layer(layerName).output)
mymodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
_________________________________________________________________


In [22]:
from keras import applications
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization

In [23]:
model= models.Sequential()
model.add(mymodel)
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))
model.layers[0].trainable = False

# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 16, 16, 64)        38720     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        9232      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [25]:
from tensorflow.keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [26]:
filepath="/content/drive/My Drive/Models/VGG16/weights-{epoch:02d}-{accuracy:.3f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

In [11]:
tf.config.experimental_run_functions_eagerly(True)


In [27]:
history = model.fit(trainX,trainY,verbose=1,epochs=20,validation_data=(testX,testY),callbacks=[checkpoint])

Train on 44964 samples, validate on 11241 samples
Epoch 1/20
44964/44964 [==============================] - 38s 845us/step - loss: 1.0581 - accuracy: 0.5999 - val_loss: 0.9323 - val_accuracy: 0.6351

Epoch 00001: accuracy improved from -inf to 0.59990, saving model to /content/drive/My Drive/Models/VGG16/weights-01-0.600.hdf5
Epoch 2/20
44964/44964 [==============================] - 37s 826us/step - loss: 0.9094 - accuracy: 0.6342 - val_loss: 1.1659 - val_accuracy: 0.6384

Epoch 00002: accuracy improved from 0.59990 to 0.63420, saving model to /content/drive/My Drive/Models/VGG16/weights-02-0.634.hdf5
Epoch 3/20
44964/44964 [==============================] - 38s 842us/step - loss: 0.8812 - accuracy: 0.6425 - val_loss: 0.9535 - val_accuracy: 0.6322

Epoch 00003: accuracy improved from 0.63420 to 0.64254, saving model to /content/drive/My Drive/Models/VGG16/weights-03-0.643.hdf5
Epoch 4/20
44964/44964 [==============================] - 37s 826us/step - loss: 0.9280 - accuracy: 0.6267 - v

In [34]:
print(model.history.history.keys())

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])


In [42]:
import json
history_dict = model.history.history
json.dump(str(history_dict), open('/content/drive/My Drive/Models/VGG16/History.json', 'w'))

In [ ]:

from keras.models import load_model
model = load_model("/content/drive/My Drive/Models/VGG16/weights-20-0.695.hdf5")
model.fit(trainX,trainY,verbose=1,epochs=50,validation_data=(testX,testY),callbacks=[checkpoint])

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Train on 44964 samples, validate on 11241 samples
Epoch 1/50
42368/44964 [===========================>..] - ETA: 7s - loss: 0.7792 - accuracy: 0.6852